### 使用DeepSeek api 读取使用 MinerU 转化后的 md 文件，然后按照：
### **期货名称-涨跌情况-涨跌原因**（三元组）形式进行分析、解读、存储

简单测试一下

In [8]:
import os
import requests
import json
import pandas as pd
from tqdm import tqdm

# 定义 API 相关信息
url = "https://swarm.wenhua.com.cn/aiservice/api/ShiXi/GetContentDeepSeekV3"
headers = {"Content-Type": "application/json"}

# 定义要处理的文件夹路径
future_folder_path = "../output_md/20250401/宝城期货_323209.pdf-8e577ede-c87b-48ec-a3c5-e01c541abbd4"
full_md_path = os.path.join(future_folder_path, "full.md")
no_info_records = []

if os.path.exists(full_md_path):
    # 读取 full.md 文件内容
    with open(full_md_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # 准备请求数据
    data = {"content": f"请总结以下内容中出现的期货名称和对应的涨跌情况（震荡/看好/看空），格式为：期货品种-跌涨情况-核心驱动因素（如果有，没有就表示为 null），注意你只需要生成期货种类-跌涨情况-核心驱动因素（比如：玉米-震荡-购入者增加），如果我的文字中没有任何期货信息，那就不返回任何东西，以下为内容：{content}"}
    
    try:
        # 调用 API
        print(f"开始处理 {full_md_path}")
        response = requests.post(url, json=data, headers=headers, stream=True)
        full_response = ""
        
        for chunk in response.iter_lines():
            if chunk:
                decoded = chunk.decode('utf-8')
                if decoded.startswith("data:"):
                    json_str = decoded[5:].strip()
                    resp_obj = json.loads(json_str)
                    content_chunk = resp_obj["choices"][0]["delta"].get("content", "")
                    full_response += content_chunk
                    if resp_obj["choices"][0].get("finish_reason") == "stop":
                        break
        
        if "-" in full_response:
            # 生成 TXT 文件名
            dir_name = os.path.basename(future_folder_path)
            txt_filename = dir_name.split(".pdf")[0] + ".txt"
            txt_path = os.path.join(future_folder_path, txt_filename)
            
            # 写入总结结果
            with open(txt_path, "w", encoding="utf-8") as txt_f:
                txt_f.write(full_response)
            print(f"总结结果已保存到 {txt_path}")
        else:
            no_info_records.append(future_folder_path)
            print(f"{full_md_path} 中未找到期货信息")
    except Exception as e:
        print(f"调用 API 处理 {full_md_path} 时出错: {e}")
else:
    print(f"未找到文件 {full_md_path}")

# 将无信息记录存入 XLSX 文件
if no_info_records:
    df = pd.DataFrame(no_info_records, columns=["无期货信息的文件夹路径"])
    xlsx_path = os.path.join(os.path.dirname(future_folder_path), "no_futures_info.xlsx")
    df.to_excel(xlsx_path, index=False)
    print(f"无期货信息的文件夹路径已记录到 {xlsx_path}")

开始处理 ../output_md/20250401/宝城期货_323209.pdf-8e577ede-c87b-48ec-a3c5-e01c541abbd4\full.md
总结结果已保存到 ../output_md/20250401/宝城期货_323209.pdf-8e577ede-c87b-48ec-a3c5-e01c541abbd4\宝城期货_323209.txt


### 以下代码的输出显示不全，自行将所有md文件转化即可

In [2]:
import os
import requests
import json
import pandas as pd
from tqdm import tqdm

# 定义 API 相关信息
url = "https://swarm.wenhua.com.cn/aiservice/api/ShiXi/GetContentDeepSeekV3"
headers = {"Content-Type": "application/json"}

# 定义要处理的日期范围，仅测试 20250401
date_range = ["20250414"]
output_md_folder = os.path.join(os.getcwd(), "../output_md")
no_info_records = []

# 遍历日期文件夹
for date_folder in date_range:
    date_path = os.path.join(output_md_folder, date_folder)
    if os.path.exists(date_path):
        # 获取所有期货文件夹
        future_folders = []
        for root, dirs, files in os.walk(date_path):
            for dir_name in dirs:
                future_folders.append(os.path.join(root, dir_name))
        
        # 使用 tqdm 显示进度条
        for future_folder_path in tqdm(future_folders, desc=f"处理 {date_folder} 文件夹"):
            full_md_path = os.path.join(future_folder_path, "full.md")
            if os.path.exists(full_md_path):
                # 读取 full.md 文件内容
                with open(full_md_path, "r", encoding="utf-8") as f:
                    content = f.read()
                
                # 准备请求数据
                data = {"content": f"请总结以下内容中出现的期货名称和对应的涨跌情况（震荡/看好/看空），格式为：期货品种-跌涨情况-核心驱动因素（如果有，没有就表示为 null），注意你只需要生成期货种类-跌涨情况-核心驱动因素（比如：玉米-震荡-购入者增加），如果我的文字中没有任何期货信息，那就不返回任何东西，以下为内容：{content}"}
                
                try:
                    # 调用 API
                    response = requests.post(url, json=data, headers=headers, stream=True)
                    full_response = ""
                    
                    for chunk in response.iter_lines():
                        if chunk:
                            decoded = chunk.decode('utf-8')
                            if decoded.startswith("data:"):
                                json_str = decoded[5:].strip()
                                resp_obj = json.loads(json_str)
                                content_chunk = resp_obj["choices"][0]["delta"].get("content", "")
                                full_response += content_chunk
                                if resp_obj["choices"][0].get("finish_reason") == "stop":
                                    break
                    
                    if "-" in full_response:
                        # 生成 TXT 文件名，修正此处使用 future_folder_path 中的文件夹名
                        dir_name = os.path.basename(future_folder_path)
                        txt_filename = dir_name.split(".pdf")[0] + ".txt"
                        txt_path = os.path.join(future_folder_path, txt_filename)
                        
                        # 写入总结结果
                        with open(txt_path, "w", encoding="utf-8") as txt_f:
                            txt_f.write(full_response)
                    else:
                        no_info_records.append(future_folder_path)
                except Exception as e:
                    print(f"调用 API 处理 {full_md_path} 时出错: {e}")

# 将无信息记录存入 XLSX 文件
if no_info_records:
    df = pd.DataFrame(no_info_records, columns=["无期货信息的文件夹路径"])
    xlsx_path = os.path.join(output_md_folder, "no_futures_info.xlsx")
    df.to_excel(xlsx_path, index=False)
    print(f"无期货信息的文件夹路径已记录到 {xlsx_path}")

处理 20250414 文件夹: 100%|██████████| 523/523 [18:38<00:00,  2.14s/it]  

无期货信息的文件夹路径已记录到 h:\project\economy\src\../output_md\no_futures_info.xlsx


In [1]:
import os
import requests
import json
import pandas as pd
from tqdm import tqdm  # 引入 tqdm 库

# 定义 API 相关信息
url = "https://swarm.wenhua.com.cn/aiservice/api/ShiXi/GetContentDeepSeekV3"
headers = {"Content-Type": "application/json"}

# 定义要处理的日期范围
date_range = [f"202504{str(i).zfill(2)}" for i in range(21, 30)]
output_md_folder = os.path.join(os.getcwd(), "../output_md")

for date_folder in date_range:
    print(f"正在处理日期文件夹: {date_folder}")
    date_path = os.path.join(output_md_folder, date_folder)
    no_info_records = []  # 将无信息记录列表放在每个日期文件夹循环内
    if os.path.exists(date_path):
        # 收集所有需要处理的期货文件夹
        future_folders = []
        for root, dirs, files in os.walk(date_path):
            for dir_name in dirs:
                future_folder_path = os.path.join(root, dir_name)
                full_md_path = os.path.join(future_folder_path, "full.md")
                if os.path.exists(full_md_path):
                    future_folders.append(future_folder_path)

        # 使用 tqdm 显示期货文件夹遍历进度条
        for future_folder_path in tqdm(future_folders, desc=f"处理 {date_folder} 内部文件进度"):
            full_md_path = os.path.join(future_folder_path, "full.md")
            # 读取 full.md 文件内容
            with open(full_md_path, "r", encoding="utf-8") as f:
                content = f.read()
            
            # 准备请求数据
            data = {"content": f"请总结以下内容中出现的期货名称和对应的涨跌情况（震荡/看好/看空），格式为：期货品种-跌涨情况-核心驱动因素（如果有，没有就表示为 null），注意你只需要生成期货种类-跌涨情况-核心驱动因素（比如：玉米-震荡-购入者增加），如果我的文字中没有任何期货信息，那就不返回任何东西，以下为内容：{content}"}
            
            try:
                # 调用 API
                response = requests.post(url, json=data, headers=headers, stream=True)
                full_response = ""
                
                for chunk in response.iter_lines(): 
                    if chunk:
                        decoded = chunk.decode('utf-8')
                        if decoded.startswith("data:"):
                            json_str = decoded[5:].strip()
                            resp_obj = json.loads(json_str)
                            content_chunk = resp_obj["choices"][0]["delta"].get("content", "")
                            full_response += content_chunk
                            if resp_obj["choices"][0].get("finish_reason") == "stop":
                                break
                
                if "-" in full_response:
                    # 生成 TXT 文件名
                    dir_name = os.path.basename(future_folder_path)
                    txt_filename = dir_name.split(".pdf")[0] + ".txt"
                    txt_path = os.path.join(future_folder_path, txt_filename)
                    
                    # 写入总结结果
                    with open(txt_path, "w", encoding="utf-8") as txt_f:
                        txt_f.write(full_response)
                else:
                    no_info_records.append(future_folder_path)
            except Exception as e:
                print(f"调用 API 处理 {full_md_path} 时出错: {e}")

        # 将无信息记录存入对应日期的 XLSX 文件
        if no_info_records:
            df = pd.DataFrame(no_info_records, columns=["无期货信息的文件夹路径"])
            xlsx_path = os.path.join(date_path, f"{date_folder}_no_futures_info.xlsx")
            df.to_excel(xlsx_path, index=False)
            print(f"{date_folder} 无期货信息的文件夹路径已记录到 {xlsx_path}")

正在处理日期文件夹: 20250421


处理 20250421 内部文件进度: 100%|██████████| 309/309 [17:49<00:00,  3.46s/it] 


20250421 无期货信息的文件夹路径已记录到 h:\project\economy\src\../output_md\20250421\20250421_no_futures_info.xlsx
正在处理日期文件夹: 20250422


处理 20250422 内部文件进度: 100%|██████████| 185/185 [11:57<00:00,  3.88s/it]


20250422 无期货信息的文件夹路径已记录到 h:\project\economy\src\../output_md\20250422\20250422_no_futures_info.xlsx
正在处理日期文件夹: 20250423


...

正在处理日期文件夹: 20250428


处理 20250428 内部文件进度: 100%|██████████| 298/298 [21:23<00:00,  4.31s/it]  


20250428 无期货信息的文件夹路径已记录到 h:\project\economy\src\../output_md\20250428\20250428_no_futures_info.xlsx
正在处理日期文件夹: 20250429


处理 20250429 内部文件进度: 100%|██████████| 92/92 [05:20<00:00,  3.49s/it]

20250429 无期货信息的文件夹路径已记录到 h:\project\economy\src\../output_md\20250429\20250429_no_futures_info.xlsx


In [1]:
import os
import shutil

# 定义源文件夹、目标文件夹和日期范围
output_md_folder = os.path.join(os.getcwd(), "../output_md")
output_final_folder = os.path.join(os.getcwd(), "../output_final")
date_range = [f"202504{str(i).zfill(2)}" for i in range(1, 30)]

# 遍历每个日期文件夹
for date_folder in date_range:
    source_date_path = os.path.join(output_md_folder, date_folder)
    target_date_path = os.path.join(output_final_folder, date_folder)

    # 检查源日期文件夹是否存在
    if os.path.exists(source_date_path):
        # 创建目标日期文件夹
        os.makedirs(target_date_path, exist_ok=True)

        # 遍历源日期文件夹及其子文件夹，查找 txt 文件
        for root, dirs, files in os.walk(source_date_path):
            for file in files:
                if file.endswith(".txt"):
                    source_file_path = os.path.join(root, file)
                    target_file_path = os.path.join(target_date_path, file)
                    # 复制 txt 文件到目标日期文件夹
                    shutil.copy2(source_file_path, target_file_path)
                    print(f"已复制文件: {source_file_path} -> {target_file_path}")
print("所有 txt 文件复制完成。")

已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323359_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323359_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323360_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323360_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323361_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323361_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323362_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323362_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323363_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323363_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323364_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323364_0.txt
已复制文件: h:\project\economy\src\../output_md\20250401\紫金天风_323365_0.txt -> h:\project\economy\src\../output_final\20250401\紫金天风_323365_0.txt
已复制文件: h:\project\economy\s

### 处理“紫金天风”

In [2]:
import os
import requests
import json
from tqdm import tqdm

# 从 deepseek.ipynb 中获取 API 相关信息
url = "https://swarm.wenhua.com.cn/aiservice/api/ShiXi/GetContentDeepSeekV3"
headers = {"Content-Type": "application/json"}

# 定义要处理的文件夹路径
output_final_folder = os.path.join(os.getcwd(), "../output_final")

# 遍历 output_final 下的所有日期文件夹
if os.path.exists(output_final_folder):
    for root, dirs, files in os.walk(output_final_folder):
        # 收集所有前缀为“紫金天风”的 txt 文件
        zjtf_txt_files = [os.path.join(root, file) for file in files 
                        if file.endswith(".txt") and file.startswith("紫金天风")]

        # 使用 tqdm 显示进度条
        for txt_file_path in tqdm(zjtf_txt_files, desc="处理紫金天风 TXT 文件进度"):
            try:
                # 读取 txt 文件内容
                with open(txt_file_path, "r", encoding="utf-8") as f:
                    content = f.read()

                # 准备请求数据
                data = {"content": f"请总结以下内容中出现的期货名称和对应的涨跌情况（震荡/看好/看空），格式为：期货品种-跌涨情况-核心驱动因素（如果有，没有就表示为 null），注意你只需要生成期货种类-跌涨情况-核心驱动因素（比如：玉米-震荡-购入者增加），如果我的文字中没有任何期货信息，那就不返回任何东西，以下为内容：{content}"}

                # 调用 API
                response = requests.post(url, json=data, headers=headers, stream=True)
                full_response = ""

                for chunk in response.iter_lines(): 
                    if chunk:
                        decoded = chunk.decode('utf-8')
                        if decoded.startswith("data:"):
                            json_str = decoded[5:].strip()
                            resp_obj = json.loads(json_str)
                            content_chunk = resp_obj["choices"][0]["delta"].get("content", "")
                            full_response += content_chunk
                            if resp_obj["choices"][0].get("finish_reason") == "stop":
                                break

                # 将 API 返回的结果写回原文件
                if full_response:
                    with open(txt_file_path, "w", encoding="utf-8") as f:
                        f.write(full_response)
                    print(f"已更新文件: {txt_file_path}")
            except Exception as e:
                print(f"处理文件 {txt_file_path} 时出错: {e}")
else:
    print(f"未找到文件夹: {output_final_folder}")

处理紫金天风 TXT 文件进度: 0it [00:00, ?it/s]
处理紫金天风 TXT 文件进度:  14%|█▍        | 1/7 [00:06<00:36,  6.14s/it]

已更新文件: h:\project\economy\src\../output_final\20250401\紫金天风_323359_0.txt


处理紫金天风 TXT 文件进度:  29%|██▊       | 2/7 [00:07<00:17,  3.42s/it]

已更新文件: h:\project\economy\src\../output_final\20250401\紫金天风_323360_0.txt


处理紫金天风 TXT 文件进度:  43%|████▎     | 3/7 [00:09<00:10,  2.60s/it]

...

已更新文件: h:\project\economy\src\../output_final\20250429\紫金天风_332985_0.txt


处理紫金天风 TXT 文件进度:  92%|█████████▏| 11/12 [00:18<00:01,  1.59s/it]

已更新文件: h:\project\economy\src\../output_final\20250429\紫金天风_332986_0.txt


处理紫金天风 TXT 文件进度: 100%|██████████| 12/12 [00:19<00:00,  1.65s/it]

已更新文件: h:\project\economy\src\../output_final\20250429\紫金天风_332987_0.txt


### 检测三元组的规范性，并进行清洗

In [6]:
import os
import re

# 定义源文件夹路径
source_folder = os.path.join(os.getcwd(), "../output_final")

# 定义正则表达式模式来匹配三元组信息
pattern = re.compile(r'(.+?)-(.+?)-(.+)')

# 用于存储所有三元组的第二个元素及其出现次数
second_elements_count = {} 

if os.path.exists(source_folder):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.txt'):
                source_file_path = os.path.join(root, file)
                try:
                    # 读取源文件内容
                    with open(source_file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                    # 提取三元组信息
                    lines = content.splitlines()
                    for line in lines:
                        # 尝试匹配三元组信息，忽略格式标记
                        cleaned_line = re.sub(r'[\d.\*\s]', '', line)
                        match = pattern.search(cleaned_line)
                        if match:
                            second_element = match.group(2)
                            # 更新元素出现次数
                            second_elements_count[second_element] = second_elements_count.get(second_element, 0) + 1
                except Exception as e:
                    print(f"处理文件 {source_file_path} 时出错: {e}")
    # 输出所有不同的第二个元素及其出现次数
    print("所有三元组的第二个元素出现次数统计如下：")
    for element, count in sorted(second_elements_count.items()):
        print(f"{element}: {count}")
else:
    print(f"未找到源文件夹: {source_folder}")

所有三元组的第二个元素出现次数统计如下：
%): 17
%、: 1
%）: 5
-正套持有: 1
BZ价差: 1
Brent: 1
C: 2
CF: 4
EC: 6
EC价差: 2
IH强于IM（分化）: 1
MEG、EB: 1
MEG价差: 1
PX价差: 1
PX价差和: 1
SC: 1
SR: 4
Sc: 1
T: 7
TF: 7
TL: 7
TS: 7
UR: 3
WTI: 1
c: 4
lh: 3
m: 4
null: 121
p: 4
v: 2
y: 4
上涨: 650
上涨%: 3
上涨动力不足（短期调整风险）: 1
上涨受限: 1
上涨趋势: 12
上涨驱动力不足: 1
上涨（%）: 7
上涨（+%）: 16
上涨（但受工业属性压制）: 1
上涨（周涨%）: 3
上涨（增仓上行）: 1
上涨（情绪推动）: 1
上涨（短期）/震荡（中期）: 1
上涨（远月合约）: 1
上行承压: 5
下挫: 3
下挫震荡: 1
下方支撑偏强: 1
下方有限: 2
下方空间有限: 1
下滑: 1
下行: 7
下跌: 910
下跌有限: 3
下跌空间有限: 15
下跌空间有限（震荡）: 1
下跌趋势: 10
下跌风险: 1
下跌（: 18
下跌（%）: 2
下跌（增仓下行）: 2
下跌（表格数据显示最新价下跌%）: 1
下跌（跌幅%）: 1
下降: 1
中性: 8
中性偏多: 2
中性偏空: 6
中期偏空: 1
中长期上行: 1
中长期逢高沽空: 1
产能利用率: 2
产量: 2
价格偏弱运行: 1
价格承压: 1
价格松动: 1
企稳: 1
企稳回升: 3
企稳运行: 1
估值修复: 1
低位企稳: 1
低位反弹: 1
低位整理: 9
低位波动: 2
低位运行: 5
低位运行维持: 2
低位震荡: 51
低位震荡调整: 1
低位高波震荡: 1
低开回升: 1
低硫弱高硫强: 1
偏下行: 1
偏多: 65
偏多头突破上涨: 6
偏多头突破上涨后高位回落: 1
偏多头趋势高位震荡: 2
偏多头高位震荡: 1
偏多调整: 1
偏弱: 42
偏弱下行: 1
偏弱势: 1
偏弱整理: 2
偏弱调整: 1
偏弱走势: 2
偏弱运行: 17
偏弱震荡: 125
偏弱高波运行: 1
偏强: 28
偏强运行: 6
偏强震荡: 57
偏空: 80
先跌后涨: 1
关注上方压力: 2
关注

In [10]:
import os
import re

# 定义源文件夹、目标文件夹路径
source_folder = os.path.join(os.getcwd(), "../output_final")
target_folder = os.path.join(os.getcwd(), "../output_final_cleaned")

# 读取 2.txt 文件，构建映射关系
classification_file = os.path.join(os.getcwd(), "../2.txt")
item_to_category = {}

try:
    with open(classification_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if ":" in line:
                item, category = line.split(":", 1)
                item = item.strip()
                category = category.strip()
                item_to_category[item] = category
except FileNotFoundError:
    print(f"未找到分类文件: {classification_file}")

# 定义正则表达式模式来匹配三元组信息
pattern = re.compile(r'(.+?)-(.+?)-(.+)')

if os.path.exists(source_folder):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.txt'):
                source_file_path = os.path.join(root, file)
                # 计算目标文件路径
                relative_path = os.path.relpath(source_file_path, source_folder)
                target_file_path = os.path.join(target_folder, relative_path)
                
                # 创建目标文件夹
                target_dir = os.path.dirname(target_file_path)
                os.makedirs(target_dir, exist_ok=True)

                try:
                    # 读取源文件内容
                    with open(source_file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                    # 提取三元组信息并过滤
                    lines = content.splitlines()
                    valid_lines = []
                    for line in lines:
                        # 尝试匹配三元组信息，忽略格式标记
                        cleaned_line = re.sub(r'[\d.\*\s]', '', line)
                        match = pattern.search(cleaned_line)
                        if match:
                            second_element = match.group(2)
                            # 检查是否能在映射中找到且分类不为 null
                            if second_element in item_to_category and item_to_category[second_element] != "null":
                                # 改写三元组，添加分类信息
                                category = item_to_category[second_element]
                                new_line = f"{match.group(1)}-{category}-{match.group(3)}"
                                valid_lines.append(new_line)

                    # 如果清洗后文件不为空，则写入目标文件
                    if valid_lines:
                        with open(target_file_path, 'w', encoding='utf-8') as f:
                            f.write('\n'.join(valid_lines))
                        print(f"已处理文件: {source_file_path} -> {target_file_path}")
                    else:
                        print(f"文件 {source_file_path} 清洗后为空，不写入新文件")
                except Exception as e:
                    print(f"处理文件 {source_file_path} 时出错: {e}")
else:
    print(f"未找到源文件夹: {source_folder}")

已处理文件: h:\project\economy\src\../output_final\20250401\东海期货_323471.txt -> h:\project\economy\src\../output_final_cleaned\20250401\东海期货_323471.txt
已处理文件: h:\project\economy\src\../output_final\20250401\中衍期货_323136_0.txt -> h:\project\economy\src\../output_final_cleaned\20250401\中衍期货_323136_0.txt
已处理文件: h:\project\economy\src\../output_final\20250401\中衍期货_323137_0.txt -> h:\project\economy\src\../output_final_cleaned\20250401\中衍期货_323137_0.txt
已处理文件: h:\project\economy\src\../output_final\20250401\五矿期货_323248.txt -> h:\project\economy\src\../output_final_cleaned\20250401\五矿期货_323248.txt
已处理文件: h:\project\economy\src\../output_final\20250401\五矿期货_323249.txt -> h:\project\economy\src\../output_final_cleaned\20250401\五矿期货_323249.txt
已处理文件: h:\project\economy\src\../output_final\20250401\五矿期货_323250.txt -> h:\project\economy\src\../output_final_cleaned\20250401\五矿期货_323250.txt
已处理文件: h:\project\economy\src\../output_final\20250401\五矿期货_323251.txt -> h:\project\economy\src\../output_final_cle